In [24]:
import numpy as np
import pandas as pd
import os
import tensorflow as tf

from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras import Model, layers, Sequential, optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint

import matplotlib.pyplot as plt

from pprint import pprint

In [18]:
# Paths

local_weights_file = './inception_v3_weights.h5'

train_path = "C:/Users/omkar/Downloads/dataset/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/train"
valid_path = "C:/Users/omkar/Downloads/dataset/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/valid"
test_path = "C:/Users/omkar/Downloads/dataset/test"

In [19]:
training_images = []

for root, dirs, files in os.walk(train_path):
    for file in files:
        training_images.append(os.path.join(root, file))
                               
validation_images = []

for root, dirs, files in os.walk(valid_path):
    for file in files:
        validation_images.append(os.path.join(root, file))

In [20]:
print("Training:")
print("Training Path:" + train_path)
print("Training Classes:" + str(len(os.listdir(train_path))))
print("Training Images:" + str(len(training_images)))

print("\n")

print("Validation:")
print("Validation Path:" + valid_path)
print("Validation Classes:" + str(len(os.listdir(valid_path))))
print("Validation Images:" + str(len(validation_images)))

print("\n")

print("Testing:")
print("Testing Path:" + test_path)
print("Testing Images:" + str(len(os.listdir(os.path.join(test_path, 'test')))))

Training:
Training Path:C:/Users/omkar/Downloads/dataset/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/train
Training Classes:38
Training Images:70295


Validation:
Validation Path:C:/Users/omkar/Downloads/dataset/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/valid
Validation Classes:38
Validation Images:17572


Testing:
Testing Path:C:/Users/omkar/Downloads/dataset/test
Testing Images:33


In [21]:
# Data Prep

train_datagen = ImageDataGenerator(
    rescale=1/255,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.2,
    shear_range=0.2,
    fill_mode='nearest'
)

validation_datagen = ImageDataGenerator(rescale=1/255)

test_datagen = ImageDataGenerator(rescale=1/255)

In [22]:
batch_size = 32

In [23]:
train_generator = train_datagen.flow_from_directory(
    train_path,
    batch_size = batch_size,
    class_mode = 'categorical',
    target_size = (150, 150),
    color_mode="rgb",
    shuffle=True
)

validation_generator =  validation_datagen.flow_from_directory(
    valid_path,
    batch_size  = batch_size,
    class_mode  = 'categorical',
    target_size = (150, 150),
    color_mode="rgb",
    shuffle=True
)

test_generator = test_datagen.flow_from_directory(
    test_path,
    batch_size  = 1,
    class_mode  = None,
    target_size = (150, 150),
    color_mode="rgb",
    shuffle=False
)

Found 70295 images belonging to 38 classes.
Found 17572 images belonging to 38 classes.
Found 33 images belonging to 1 classes.


In [27]:
train_num = train_generator.samples
valid_num = validation_generator.samples

In [28]:
 # Moedel
    
# Creating Instence of pre-trained model from Keras Application
inception_model = InceptionV3(input_shape= (150, 150, 3),
                                include_top = False,
                                weights = None)

inception_model.load_weights(local_weights_file)

In [29]:
# Freezing all Layers of Inception V3 Model
for layer in inception_model.layers:
    layer.trainable = False

In [30]:
inception_model.summary()

Model: "inception_v3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 150, 150, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 74, 74, 32)   864         ['input_1[0][0]']                
                                                                                                  
 batch_normalization (BatchNorm  (None, 74, 74, 32)  96          ['conv2d[0][0]']                 
 alization)                                                                                       
                                                                                       

In [31]:
# Taking output from 'mixed8' layer
last_layer = inception_model.get_layer('mixed9')
print('Last Layer Output Shape:', last_layer.output_shape)
last_output = last_layer.output

Last Layer Output Shape: (None, 3, 3, 2048)


In [32]:
# Adding our own layers at the end of Inception Network

# Flatten the output layer to 1 dimension
x = layers.Flatten()(last_output)

# Add a fully connected layer with 1024 hidden units and ReLU activation
x = layers.Dense(2048, activation='relu')(x)

# Add a fully connected layer with 1024 hidden units and ReLU activation
x = layers.Dense(1024, activation='relu')(x)

# Add a dropout rate of 0.2
x = layers.Dropout(0.2)(x)

# Add a final sigmoid layer for classification
x = layers.Dense(len(class_dict), activation='softmax')(x)

model = Model(inception_model.input, x)

In [38]:
model.compile(optimizer=optimizers.RMSprop(learning_rate=0.0001),
              loss = 'categorical_crossentropy',
              metrics = ['accuracy'])

In [39]:
history = model.fit(
    train_generator,
    steps_per_epoch=train_num//batch_size,
    validation_data = validation_generator,
    validation_steps=valid_num//batch_size,
    epochs = 5,
#     callbacks=callbacks_list,
    verbose = 1
)

Epoch 1/5
 383/2196 [====>.........................] - ETA: 35:32 - loss: 0.6070 - accuracy: 0.8077

KeyboardInterrupt: 